<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/2021notebooks/2021_1010facial_keypoints_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

---
title: Getting Started with Facial Keypoint Detection using Deep Learning and PyTorch
author: Sovit Ranjan 
date: 2020October
source: https://debuggercafe.com/getting-started-with-facial-keypoint-detection-using-pytorch/

---

- [Kaggle](https://www.kaggle.com/c/facial-keypoints-detection/data)
- Kaggle からデータ入手

```bash
kaggle competitions download -c facial-keypoints-detection
```

In [ ]:
%%shell
curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=1r1XxfBOQMzfhaohgg6aq-KbsYACkiWNa" > /dev/null
CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"  
curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=1r1XxfBOQMzfhaohgg6aq-KbsYACkiWNa" -o kaggle_facial_keypoints_detection.tar.gz


In [ ]:
!gunzip kaggle_facial_keypoints_detection.tar.gz
!tar -xf kaggle_facial_keypoints_detection.tar

# 深層学習とPyTorchを使った顔のキーポイント検出

PyTorch を使った顔のキーポイント検出デモ

<center>
<img src="https://debuggercafe.com/wp-content/uploads/2020/10/intro_exmp.png" width="33%"><br/>
<p style="text-align:left;width:88%; background-color:cornsilk">
Figure 1. An example of facial keypoint detection using deep learning and PyTorch. We will try to achieve similar results after going through this tutorial.
</p>
</center>

図 1 は濃淡画像上での顔のキーポイント検出の例です。
このチュートリアルの終わりまでに， 同様の結果を得ることを目標としています。

* 顔のキーポイント検出の必要性を簡単に紹介します。
* 深層学習と PyTorch を使って、顔のキーポイント検出を始めるための簡単なデータセットを使用しています。
* シンプルな畳み込みニューラルネットワークモデルを使って、データセット上で訓練を行います。
* 次に、訓練されたモデルを使って、テストデータセットの未見の画像の顔のキーポイントを検出します。
* 最後に、メリット、デメリット、さらなる実験と改善のために取るべきステップにたどり着きます。

# 1. なぜ顔のキーポイント検出が必要なのか？


先に進む前に、素朴な疑問に答えてみましょう。
なぜ、顔のキーポイント検出のような技術が必要なのか？たくさんありますが、いくつかをご紹介します。

スマートフォンのアプリで， フィルターを見たことがある人も多いのではないでしょうか。
このようなフィルターを顔に正確に適用するためには，人の顔のキーポイント (注目点) を正しく判断する必要があります。
そのためには， 顔のキーポイントを検出する必要があります。
また、顔のキーポイント検出は、人の年齢を判定するのにも利用できます。
実際、多くの産業や企業がこの技術を利用しています。
顔認証によるスマートフォンのロック解除にも、顔のキーポイント検出が使われています。
上記は、実際の使用例の一部に過ぎません。
他にもたくさんありますが、それらの詳細については今は触れません。
もっと詳しく知りたい方は、ユースケースについてより多くのポイントを説明した[こちらの記事をお読みください](https://www.facefirst.com/blog/amazing-uses-for-face-recognition-facial-recognition-use-cases/)。

上述したように、このチュートリアルでは、顔のキーポイント検出にディープラーニングを使用します。
深層学習と畳み込みニューラルネットワークは、現在、顔認識とキーポイント検出の分野で大きな役割を果たしています。

## 1.1 データセット
<!-- ## 1.1 The Dataset-->

過去に開催された Kaggle のコンペティションのデータセットを使用します。
競技名は Facial Keypoints Detection です。
競技のルールに同意した後、データセットをダウンロードするように言われたら、ダウンロードしてください。

データセットは大きくありません。約 80 MBしかありません。
訓練データセットとテストデータセットを含む CSV ファイルで構成されています。
画像も CSV ファイルの中にピクセル値で入っています。
画像はすべて 96×96 次元の濃淡画像です。
濃淡画像で次元が小さいため、深層学習による顔のキーポイント検出を始めるのに適した、簡単なデータセットです。

このデータセットには (x, y) 形式の 15 個の座標特徴のキーポイントが含まれています。
つまり、各顔画像には合計 30 個のポイント特徴があるということです。
すべてのデータポイントは CSV ファイルの異なる列に入っており、最後の列には画像のピクセル値が入っています。

<!--
We will use a dataset from one of the past Kaggle competitions. 
The competition is Facial Keypoints Detection. Go ahead and download the dataset after accepting the competition rules if it asks you to do so.

The dataset is not big. It is only around 80 MB. 
It consists of CSV files containing the training and test dataset. 
The images are also within the CSV files with the pixel values. 
All the images are 96×96 dimensional grayscale images. 
As the images are grayscale and small in dimension, that is why it is a good and easy dataset to start with facial keypoint detection using deep learning.

The dataset contains the keypoints for 15 coordinate features in the form of (x, y). 
So, there are a total of 30 point features for each face image. 
All the data points are in different columns of the CSV file with the final column holding the image pixel values.-->

次のコードスニペットは CSV ファイルのデータフォーマットを示しています。
<!-- The following code snippet shows the data format in the CSV files. -->


<pre style="background-color:powderblue">
left_eye_center_x  left_eye_center_y  right_eye_center_x  ...  mouth_center_bottom_lip_x  mouth_center_bottom
_lip_y                                              Image
0             66.033564          39.002274           30.227008  ...                  43.130707                
  84.485774  238 236 237 238 240 240 239 241 241 243 240 23...
1             64.332936          34.970077           29.949277  ...                  45.467915                
  85.480170  219 215 204 196 204 211 212 200 180 168 178 19...
2             65.057053          34.909642           30.903789  ...                  47.274947                
  78.659368  144 142 159 180 188 188 184 180 167 132 84 59 ...
3             65.225739          37.261774           32.023096  ...                  51.561183                
  78.268383  193 192 193 194 194 194 193 192 168 111 50 12 ...
4             66.725301          39.621261           32.244810  ...                  44.227141                
  86.871166  147 148 160 196 215 214 216 217 219 220 206 18...
...                 ...                ...                 ...  ...                        ...                
        ...                                                ...
7044          67.402546          31.842551           29.746749  ...                  50.426637                
  79.683921  71 74 85 105 116 128 139 150 170 187 201 209 2...
7045          66.134400          38.365501           30.478626  ...                  50.287397                
  77.983023  60 60 62 57 55 51 49 48 50 53 56 56 106 89 77 ...
7046          66.690732          36.845221           31.666420  ...                  49.462572                
  78.117120  74 74 74 78 79 79 79 81 77 78 80 73 72 81 77 1...
7047          70.965082          39.853666           30.543285  ...                  50.065186                
  79.586447  254 254 254 254 254 238 193 145 121 118 119 10...
7048          66.938311          43.424510           31.096059  ...                  45.900480                
  82.773096  53 62 67 76 86 91 97 105 105 106 107 108 112 1...</pre>


キーポイントとなる特徴的な列を見ることができます。
このような列は、顔の左側と右側で30個あります。
最後の列は、ピクセル値を示す「画像」の列です。
これは文字列形式です。
このように、データセットに深層学習技術を適用する前に、少しだけ前処理を行う必要があります。
<!-- You can see the keypoint feature columns. 
There are 30 such columns for the left and right sides of the face. 
The last column is the Image column with the pixel values. 
They are in string format. 
So, we will have to do a bit of preprocessing before we can apply our deep learning techniques to the dataset. -->


以下は、顔にキーポイントを設定した `training.csv` ファイルのサンプル画像です。
<!-- The following are some sample images from the training.csv file with the keypoints on the faces. -->

<center>
    <img src="https://debuggercafe.com/wp-content/uploads/2020/10/training_samples.png" width="66%"><br/>
<p style="text-align:left;width:77%;background-color:cornsilk">
Figure 2. Some samples from the training set with their facial keypoints. 
We will use this dataset to train our deep neural network using PyTorch.</p>
</center>

また、このデータセットには多くの欠損値が含まれています。
7048 個のインスタンス (行) のうち 4909 行は 1 つ以上の列で少なくとも 1 つの NULL 値を含んでいます。
また、全てのキーポイントが揃っている完全なデータは 2140 行のみです。
このような状況は、データセットを作成する際に対処しなければなりません。


# 2. 深層学習とPyTorchによる顔のキーポイント検出
<!-- # 2. Facial Keypoint Detection using Deep Learning and PyTorch -->

PyTorch フレームワークを使った顔のキーポイント検出のためのコーディング作業に入っていきます。

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

!pip install japanize_matplotlib
import japanize_matplotlib

In [ ]:
import torch

ROOT_PATH = 'kaggle_facial_keypoints_detection'
!mkdir outputs
OUTPUT_PATH = 'outputs'

# learning parameters
BATCH_SIZE = 256
LR = 0.0001
EPOCHS = 300

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train/test split
TEST_SPLIT = 0.2
TEST_SPLIT = 0.1

# show dataset keypoint plot
SHOW_DATASET_PLOT = True

トレーニングと検証のための学習パラメータは以下の通りです。

* バッチサイズは 256 としています。
画像のサイズが 96×96 と小さく、また濃淡画像であるため，ミニバッチサイズを大きくしてもメモリの問題は発生しません。
ただし GPU のメモリに応じて、バッチサイズを自由に増減させてください。
* 学習率は 0.0001 です。
様々な学習率を試した結果、今回使用するモデルとデータセットでは、この学習率が最も安定していると思われます。
* 顔のキーポイントのデータセットに対して 300 エポックでモデルを学習します。
多いと思われるかもしれませんが、実際には， このように多くのエポックを行うことで， モデルは恩恵を受けます。
* 0.2 のテスト分割を使用しています。
 データの 80 %をトレーニングに，20 %を検証に使用します。
* SHOW_DATASET_PLOT が True の場合， 訓練直前に，いくつかの顔と，それに対応する顔のキーポイントのプロットが表示されます。
必要であれば，これを False にしておくこともできます。

## 2.1 深層学習と PyTorch による顔のキーポイント検出のためのユーティリティー関数の作成

この節では、作業を容易にするためのユーティリティー関数をいくつか書きます。
ユーティリティー関数は全部で 3 つあります。
この 3 つのユーティリティー関数はすべて、顔の画像上に顔のキーポイントをプロットするのに役立ちます。
しかし 3 つとも異なるシナリオに対応しています。
1 つずつ取り組んでいきましょう。

### 2.1.1 顔に検証キーポイントをプロットする関数

まず、検証用のキーポイントをプロットする関数を紹介します。
この関数を `valid_keypoints_plot()` と呼ぶことにします。
この関数は基本的に，与えられた一定のエポック数の後に，画像の顔上に検証（回帰したキーポイント）をプロットします．

まずはコードを書いてみましょう。その後、重要な部分の説明に入ります。


In [ ]:
def valid_keypoints_plot(image, outputs, orig_keypoints, epoch):
    """
    This function plots the regressed (predicted) keypoints and the actual keypoints after each validation epoch for one image in the batch.
    """
    # detach the image, keypoints, and output tensors from GPU to CPU
    image = image.detach().cpu()
    outputs = outputs.detach().cpu().numpy()
    orig_keypoints = orig_keypoints.detach().cpu().numpy()

    # just get a single datapoint from each batch
    img = image[0]
    output_keypoint = outputs[0]
    orig_keypoint = orig_keypoints[0]
    img = np.array(img, dtype=float)
    img = np.transpose(img, (1, 2, 0))
    img = img.reshape(96, 96)
    plt.imshow(img, cmap='gray')

    output_keypoint = output_keypoint.reshape(-1, 2)
    orig_keypoint = orig_keypoint.reshape(-1, 2)
    for p in range(output_keypoint.shape[0]):
        plt.plot(output_keypoint[p, 0], output_keypoint[p, 1], 'r.')
        plt.text(output_keypoint[p, 0], output_keypoint[p, 1], f"{p}")
        plt.plot(orig_keypoint[p, 0], orig_keypoint[p, 1], 'g.')
        plt.text(orig_keypoint[p, 0], orig_keypoint[p, 1], f"{p}")
    plt.savefig(f"{OUTPUT_PATH}/val_epoch_{epoch}.png")
    plt.close()


def test_keypoints_plot(images_list, outputs_list, figsize=(10,10)):
    """
    This function plots the keypoints for the outputs and images
    in `test.csv` file.
    """
    plt.figure(figsize=figsize)
    for i in range(len(images_list)):
        outputs = outputs_list[i]
        image = images_list[i]
        outputs = outputs.cpu().detach().numpy()
        outputs = outputs.reshape(-1, 2)
        plt.subplot(3, 3, i+1)
        #plt.imshow(image, cmap='gray')
        plt.imshow(image.reshape(96,96), cmap='gray')

        for p in range(outputs.shape[0]):
                plt.plot(outputs[p, 0], outputs[p, 1], 'r.')
                plt.text(outputs[p, 0], outputs[p, 1], f"{p}")
        plt.axis('off')

    plt.savefig(f"{OUTPUT_PATH}/test_output.pdf")
    plt.show()
    plt.close()


def dataset_keypoints_plot(data, figsize=(22,20), n_samples=30):
    """
    This function shows the image faces and keypoint plots that the model will actually see. 
    This is a good way to validate that our dataset is in fact corrent and the faces align wiht the keypoint features. 
    The plot will be show just before training starts. Press `q` to quit the plot and start training.
    """
    plt.figure(figsize=figsize)
    for i in range(n_samples):
        sample = data[i]
        img = sample['image']
        img = np.array(img, dtype=float)
        img = np.transpose(img, (1, 2, 0))
        img = img.reshape(96, 96)
        plt.subplot(5, 6, i+1)
        plt.imshow(img, cmap='gray')
        keypoints = sample['keypoints']
        for j in range(len(keypoints)):
            plt.plot(keypoints[j, 0], keypoints[j, 1], 'r.')
    plt.show()
    plt.close()

最初の 2 行のコメントを読めば，この関数の要点が容易に理解できると思います。
画像テンソル（`image`），出力テンソル（`outputs`），データセットからのオリジナルキーポイント（`orig_keypoints`）を，エポック番号とともに関数に渡します．
<!-- If you read the comment in the first two lines then you will easily get the gist of the function. 
We provide the image tensors (`image`), the output tensors (`outputs`), and the original keypoints from the dataset (`orig_keypoints`) along with the epoch number to the function.-->

* 7, 8, 9行目では GPU からデータを切り離し CPU にロードしています。
* テンソルは，画像，予測されたキーポイント，オリジナルのキーポイントそれぞれについて 256 個のデータポイントを含むバッチの形をしています．
12 行目から 14 行目で、それぞれの最初のデータポイントを取得します。
* 次に，画像を NumPy の配列形式に変換し，チャンネルを最後にして転置し，元の 96×96 のサイズに整形します。
そして，Matplotlib を使って画像をプロットします。
* 21 行目と 22 行目では，予測されたキーポイントと元のキーポイントの形状を変更します．
21 行目と 22 行目で，予測キーポイントとオリジナルキーポイントの形状を変更します．
* 23 行目から 27 行目まで，顔の画像上に予測キーポイントとオリジナルキーポイントをプロットします。
予測されたキーポイントは赤のドットで、オリジナルのキーポイントは緑のドットになります。
また，`plt.text()`を用いて，対応するキーポイントの番号をプロットします。
* 最後に，画像を `outputs` フォルダに保存します。

<!--
* At lines 7, 8, and 9 we detach the data from the GPU and load them onto the CPU.
* The tensors are in the form of a batch containing 256 datapoints each for the image, the predicted keypoints, and the original keypoints. 
We get just the first datapoint from each from lines 12 to 14.
* Then we convert the image to NumPy array format, transpose it make channels last, and reshape it into the original 96×96 dimensions. 
Then we plot the image using Matplotlib.
* At lines 21 and 22, we reshape the predicted and original keypoints. 
This will make them have 2 columns along with the respective number of rows.
* Starting from lines 23 till 27, we plot the predicted and original keypoints on the image of the face. 
The predicted keypoints will be red dots while the original keypoints will be green dots. 
We also plot the corresponding keypoint numbers using `plt.text()`.
* Finally, we save the image in the `outputs` folder.
-->

Now, we will move onto the next function for the `utils.py` file.

### 2.1.2 テスト用キーポイントを顔にプロットする関数
<!-- ### 2.1.2 Function to Plot the Test Keypoints on the Faces -->

ここでは、テスト時に予測するキーポイントをプロットするためのコードを書きます。
具体的には `test.csv` ファイルにピクセル値が入っている画像が対象となります。
<!--
Here, we will write the code for plotting the keypoints that we will predict during testing. 
Specifically, this is for those images whose pixel values are in the test.csv file. -->

In [ ]:
import torch
import cv2
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

resize = 96

def train_test_split(csv_path, split):
    df_data = pd.read_csv(csv_path)

    # drop all the rows with missing values
    df_data = df_data.dropna()
    len_data = len(df_data)

    # calculate the validation data sample length
    valid_split = int(len_data * split)

    # calculate the training data samples length
    train_split = int(len_data - valid_split)
    training_samples = df_data.iloc[:train_split][:]
    valid_samples = df_data.iloc[-valid_split:][:]
    print(f"訓練データサンプル数: {len(training_samples)}")
    print(f"検証データサンプル数: {len(valid_samples)}")
    return training_samples, valid_samples


class FaceKeypointDataset(Dataset):
    def __init__(self, samples):
        self.data = samples

        # get the image pixel column only
        self.pixel_col = self.data.Image
        self.image_pixels = []
        #for i in tqdm(range(len(self.data))):
        for i in range(len(self.data)):
            img = self.pixel_col.iloc[i].split(' ')
            self.image_pixels.append(img)
        self.images = np.array(self.image_pixels, dtype=float)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):

        # reshape the images into their original 96x96 dimensions
        image = self.images[index].reshape(96, 96)
        orig_w, orig_h = image.shape

        # resize the image into `resize` defined above
        image = cv2.resize(image, (resize, resize))

        # again reshape to add grayscale channel format
        image = image.reshape(resize, resize, 1)
        image = image / 255.0

        # transpose for getting the channel size to index 0
        image = np.transpose(image, (2, 0, 1))

        # get the keypoints
        keypoints = self.data.iloc[index][:30]
        keypoints = np.array(keypoints, dtype=float)

        # reshape the keypoints
        keypoints = keypoints.reshape(-1, 2)
        # rescale keypoints according to image resize
        keypoints = keypoints * [resize / orig_w, resize / orig_h]
        return {
            'image': torch.tensor(image, dtype=torch.float),
            'keypoints': torch.tensor(keypoints, dtype=torch.float),
        }


# get the training and validation data samples
training_samples, valid_samples = train_test_split(f"{ROOT_PATH}/training.csv", TEST_SPLIT)

# initialize the dataset - `FaceKeypointDataset()`
print('--- PREPARING DATA ---')
train_data = FaceKeypointDataset(training_samples)
valid_data = FaceKeypointDataset(valid_samples)
print('--- DATA PREPRATION DONE ---')
# prepare data loaders
train_loader = DataLoader(train_data,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
valid_loader = DataLoader(valid_data,
                          batch_size=BATCH_SIZE,
                          shuffle=False)


# whether to show dataset keypoint plots
if SHOW_DATASET_PLOT:
    dataset_keypoints_plot(valid_data)


In [ ]:
#dataset_keypoints_plot(valid_data,figsize=(24,20))
#dataset_keypoints_plot(train_data,figsize=(24,20))
#df_data = pd.read_csv(f"{ROOT_PATH}/training/training.csv")
#df_data

In [ ]:
#from model import FaceKeypointModel
import torch.nn as nn
import torch.nn.functional as F

class FaceKeypointModel(nn.Module):
    def __init__(self):
        super(FaceKeypointModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128, 30) 
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(p=0.2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)

        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.dropout(x)
        out = self.fc1(x) 
        return out

`test_keypoints_plot()` 関数の入力パラメータは， `images_list` と `outputs_list` です。
これらは， 指定された数の入力画像と， プロットしたい予測キーポイントを含む 2 つのリストです。
この関数は非常にシンプルです。

* 7 行目からは，単純な for ループを実行し，2 つのリストに含まれる画像と予測されるキーポイントをループします。
* `valid_keypoints_plot()` 関数と同じ経路をたどります。
* しかし，今回は，すべての画像を 1 つのプロットにしたいので，Matplotlib の `subplot()` 関数を利用します。
9 枚の画像をプロットするので，`plt.subplot(3, 3, i+1)` を使用します．
* 最後に， プロットされた画像と予測されたキーポイントを，出力フォルダに保存します。

以上でこの関数の説明は終わりです。

### 2.1.3 入力データセットの顔画像とキーポイントをプロットする関数

ニューラルネットワークモデルにデータを入力する前に，データが正しいかどうかを確認します。
すべてのキーポイントが顔に正しく対応しているかどうかは，わからないかもしれません。
そのため，学習開始前に，顔画像とそれに対応するキーポイントを表示する関数を書きます。
`SHOW_DATASET_PLOT = True` になっている場合のみ行われます。


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn as nn
import matplotlib

# model
model = FaceKeypointModel().to(DEVICE)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=LR)

# we need a loss function which is good for regression like MSELoss
criterion = nn.MSELoss()

# training function
def train(model, dataloader, data):
    #print('Training')
    model.train()
    train_running_loss = 0.0
    counter = 0
    # calculate the number of batches
    num_batches = int(len(data)/dataloader.batch_size)

    for i, data in enumerate(dataloader): #, total=num_batches):
        counter += 1
        image, keypoints = data['image'].to(DEVICE), data['keypoints'].to(DEVICE)
        # flatten the keypoints
        keypoints = keypoints.view(keypoints.size(0), -1)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, keypoints)
        train_running_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_loss = train_running_loss/counter
    return train_loss


# validatioon function
def validate(model, dataloader, data, epoch, print_interval=3):
    #print('Validating')
    model.eval()
    valid_running_loss = 0.0
    counter = 0
    # calculate the number of batches
    num_batches = int(len(data)/dataloader.batch_size)
    with torch.no_grad():
        for i, data in enumerate(dataloader): #, total=num_batches):
            counter += 1
            image, keypoints = data['image'].to(DEVICE), data['keypoints'].to(DEVICE)
            # flatten the keypoints
            keypoints = keypoints.view(keypoints.size(0), -1)
            outputs = model(image)
            loss = criterion(outputs, keypoints)
            valid_running_loss += loss.item()
            # plot the predicted validation keypoints after every...
            # ... print_interval epochs and from the first batch
            if (epoch+1) % print_interval == 0 and i == 0:
                valid_keypoints_plot(image, outputs, keypoints, epoch)

    valid_loss = valid_running_loss/counter
    return valid_loss

In [ ]:
EPOCHS

In [ ]:
EPOCHS = 64 # 時間の都合上 EPOCHS を少なくしています
interval = EPOCHS >> 3

train_loss = []
val_loss = []
for epoch in range(EPOCHS):
    train_epoch_loss = train(model, train_loader, train_data)
    val_epoch_loss = validate(model, valid_loader, valid_data, epoch, print_interval=2)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)

    if ((epoch) % interval) == 0:
        print(f"エポック {epoch+1:<4d}/{EPOCHS:<4d}", end="")
        print(f"訓練損失: {train_epoch_loss:.3f}", 
              f'検証損失: {val_epoch_loss:.3f}')

In [ ]:
# loss plots
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='blue', label='訓練損失')
plt.plot(val_loss, color='red', label='検証損失')
plt.xlabel('エポック')
plt.ylabel('損失値')
plt.legend()
plt.savefig(f"{OUTPUT_PATH}/loss.pdf")
plt.show()

In [ ]:
# 結果の保存
torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, f"{OUTPUT_PATH}/model.pth")


## 結果の再読み込みと視覚化

保存した結果を再度読み込んで表示します


In [ ]:
model = FaceKeypointModel().to(DEVICE)

# load the model checkpoint
checkpoint = torch.load(f"{OUTPUT_PATH}/model.pth")

# load model weights state_dict
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

## テストデータによる結果の視覚化

`test.csv` を読み込んで，結果を表示します

In [ ]:
model.eval()

In [ ]:
# `test.csv` ファイルの読み込み

csv_file = f"{ROOT_PATH}/test.csv"
data = pd.read_csv(csv_file)

pixel_col = data.Image
image_pixels = []
for i in range(len(pixel_col)):
    img = pixel_col[i].split(' ')
    image_pixels.append(img)

# NumPy 配列へ変換
images = np.array(image_pixels, dtype=float)

## キーポイント予測結果の視覚化

9 つのデータを視覚化します。

予測されたキーポイントを取得し， `outputs` に格納します。
各前向き処理後に， 画像と出力をそれぞれ  `images_list` と `outputs_list` に追加します。

最後に，`test_keypoints_plot()` を呼び出し，予測キーポイントを顔の画像上にプロットします。

検証結果と比較すると、テスト結果は良好に見えます。


In [ ]:
images_list, outputs_list = [], []
for i in range(9):
    with torch.no_grad():
        image = images[i]
        image = image.reshape(96, 96, 1)
        image = cv2.resize(image, (resize, resize))
        image = image.reshape(resize, resize, 1)
        orig_image = image.copy()
        image = image / 255.0
        image = np.transpose(image, (2, 0, 1))
        image = torch.tensor(image, dtype=torch.float)
        image = image.unsqueeze(0).to(DEVICE)
        
        # forward pass through the model
        outputs = model(image)
        # append the current original image
        images_list.append(orig_image)
        # append the current outputs
        outputs_list.append(outputs)
        
        
test_keypoints_plot(images_list, outputs_list, figsize=(14,14))